In [1]:
import pickle
import gseapy as gp
import pandas as pd

In [2]:
# local_stringdb = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/data/string/lfs-stringdb/'
# # load local STRING database and names
# df = pd.read_csv(local_stringdb+'9606.protein.info.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
# df['preferred_name'] = df['preferred_name'].str.upper()
# stringId2name = df.set_index('#string_protein_id')['preferred_name'].to_dict()
# name2stringId = df.set_index('preferred_name')['#string_protein_id'].to_dict()
# df = pd.read_csv(local_stringdb+'9606.protein.aliases.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
# df['alias'] = df['alias'].str.upper()
# aliases2stringId = df.set_index('alias')['#string_protein_id'].to_dict()

# file_path = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/datasets/ppi/ppi_name2stringId.pkl'
# # Serialize and save the Tensor to the file
# with open(file_path, 'wb') as file:
#     pickle.dump(name2stringId, file)
# # Close the file
# file.close()

In [2]:
ppi_aliases2stringId_path = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/datasets/ppi/ppi_aliases2stringId.pkl'
ppi_name2stringId_path = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/datasets/ppi/ppi_name2stringId.pkl'
ppi_index_path = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/datasets/ppi/ppi_index.pkl'
complex_path = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/test/complex/CORUM-Human-Complexes_r.txt'
complex_save = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/test/complex/human_complexes_node.pkl'
pathway_save = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/test/pathway_node.pkl'
disgenet_save = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/test/cad_node.pkl'

In [3]:
with open(ppi_aliases2stringId_path, 'rb') as file:
    aliases2stringId = pickle.load(file)
with open(ppi_name2stringId_path, 'rb') as file:
    name2stringId = pickle.load(file)
def convert_stringId(alias):
    try:
        stringId = name2stringId[alias]
    except:
        #print(alias, 'can\'t be converted by name2stringId! Now trying aliases2stringId.')
        try:
            stringId = aliases2stringId[alias]
        except:
            #print(alias, 'can\'t be converted by aliases2stringId! Now return None.')
            stringId = None
    #print(alias, stringId)
    return stringId
protein_names = list(aliases2stringId.keys())
protein_names.extend(list(name2stringId.keys()))

with open(ppi_index_path, 'rb') as file:
    ppi_index = pickle.load(file)
node2string = {value: key for key, value in ppi_index.items()}

humans = set(ppi_index.keys())
string2name= {value: key for key, value in name2stringId.items()}

In [5]:
caddf = pd.read_csv('/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/disease/DIS_CAD.tsv',sep='\t')
cadlist = caddf['Gene'].tolist()
group_node = []
for gene in cadlist:
    if gene in protein_names:
        stringid = convert_stringId(gene)
        if stringid in humans:
            node = ppi_index[stringid]
            group_node.append(node)
with open(disgenet_save, 'wb') as file:
    pickle.dump(group_node, file)
# Close the file
file.close()


In [7]:
len(group_node)

1535

## complexs

In [10]:
# Initialize an empty dictionary to store the data
data_dict = {}

# Read the text file and populate the dictionary
with open(complex_path, 'r') as file:
    for line_index, line_content in enumerate(file, start=1):
        # Enumerate starts index from 1
        values = line_content.strip().split('\t')  # Assuming values are separated by whitespace
        uniques = list(set(values)&set(protein_names))
        if len(uniques) > 1:
            data_dict[line_index] = uniques

complexs = dict()

for key, value in data_dict.items():
    length = len(value)
    if length <= 7:
        complexs[key] = value
len(complexs)

2350

In [11]:
complexs_id = dict()

for complex_name in complexs:
    # folder_path = 'D:/study/thesis/project/HBDM-main/ppi_results/test_results/'+complex_name
    # os.mkdir(folder_path)
    group_node = []
    for gene in complexs[complex_name]:
        if gene in protein_names:
            stringid = convert_stringId(gene)
            real_gene = string2name[stringid]
            if real_gene in humans:
                node = ppi_index[real_gene]
                group_node.append(node)
            else:
                continue

    if len(set(group_node)) > 1:
        complexs_id[complex_name] = group_node
# Serialize and save the Tensor to the file
with open(complex_save, 'wb') as file:
    pickle.dump(complexs_id, file)
# Close the file
file.close()


## pathways

In [12]:
keggs = gp.get_library(name='KEGG_2019_Human', organism='Human')
smallkegg = dict()
kegglength = []
for key, value in keggs.items():
    length = len(value)
    kegglength.append(length)
    if length <= 150:
        smallkegg[key] = value


pathway_id = dict()

for pathway in smallkegg:
    group_node = []
    for gene in smallkegg[pathway]:
        if gene in protein_names:
            stringid = convert_stringId(gene)
            real_gene = string2name[stringid]
            if real_gene in humans:
                node = ppi_index[real_gene]
                group_node.append(node)
            else:
                continue

    if len(set(group_node)) > 1:
        complexs_id[complex_name] = group_node

# Serialize and save the Tensor to the file
with open(pathway_save, 'wb') as file:
    pickle.dump(pathway_id, file)
# Close the file
file.close()